<a href="https://colab.research.google.com/github/Teja21B21A4206/FMML-IIT-LABS/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [9]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [11]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [12]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [13]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [14]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [15]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1.Does averaging the validation accuracy across multiple splits give more consistent results?

A.Yes, averaging the validation accuracy across multiple splits can provide more consistent and reliable results when evaluating the performance of a machine learning model. This technique is commonly known as cross-validation and is used to assess how well a model will generalize to an independent dataset.

Here's why averaging validation accuracy across multiple splits is beneficial:

Reduces Variance: Averaging the performance metrics over multiple splits reduces the impact of the random variability in a single train-test split. Different random splits can lead to different evaluation scores. Averaging smoothens out this variance and provides a more stable estimation of the model's performance.

Better Representation of Model Performance: Cross-validation gives a more comprehensive understanding of the model's performance across different subsets of the data. Averaging the accuracy scores obtained from multiple folds provides a more representative performance metric, as it considers the model's consistency across various data partitions.

Minimizes Bias: Averaging helps in reducing the bias that might be introduced if a single split of the data happens to be particularly easy or challenging. By considering multiple splits, you get a more balanced view of how well your model is performing on different subsets of the data.

Provides Confidence Interval: By conducting cross-validation and averaging the results, you can calculate confidence intervals around your performance metric. This gives you an idea of the range in which the true model performance is likely to lie.

Helps in Model Selection: Cross-validation is often used in model selection processes like hyperparameter tuning. Averaging the results across different folds helps in choosing the best-performing model configuration.

However, it's crucial to note that the effectiveness of cross-validation, including averaging accuracy, depends on the quality and representativeness of your data splits. Randomizing the data before splitting and ensuring that each fold is a good representation of the overall dataset are important practices in cross-validation. Additionally, the choice of the number of folds (k) in techniques like k-fold cross-validation can also influence the results.

2.Does it give more accurate estimate of test accuracy?

A.Cross-validation provides a more accurate estimate of a model's performance, but it is important to clarify that this estimate pertains to how well the model is likely to perform on unseen data. In other words, cross-validation provides a better estimate of how the model will generalize to new, unseen data from the same population as the training data.

Here's why cross-validation can give a more accurate estimate of test accuracy:

Reduces Overfitting to a Single Dataset: When you train a model and evaluate its performance on a single fixed dataset (train-test split), there's a risk of overfitting to that specific dataset. Overfitting means the model becomes too tailored to the peculiarities of the training data and might not generalize well to new, unseen data. Cross-validation, by training and evaluating the model on multiple different subsets of the data, reduces the risk of overfitting to a particular dataset.

Utilizes the Entire Dataset: Cross-validation allows every data point to be used for both training and validation exactly once. In k-fold cross-validation, the dataset is divided into k subsets, and the model is trained and validated k times, with each subset serving as the validation data exactly once. This means all data points are used for validation, providing a more comprehensive assessment of the model's performance.

Better Performance Metric: Averaging the results across multiple folds gives a more stable and reliable performance metric. It smoothens out the effects of variability in a single train-test split, providing a more accurate estimate of the model's generalization performance.

Robustness to Data Variability: If the dataset is diverse or contains outliers, different train-test splits can highlight different aspects of the data. Cross-validation, by considering multiple such splits, provides a more robust assessment of the model's performance across different subsets of the data.

While cross-validation gives a more accurate estimate of how well the model is likely to perform on new data, it's important to understand that this estimate is based on the assumption that the new data comes from the same distribution as the training data. If the new data significantly differs from the training data, the model's performance might not be accurately reflected by cross-validation results. Therefore, it's crucial to ensure that the training data is representative of the overall population the model is expected to generalize to.

3.What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?

A.The number of iterations, also known as the number of epochs, plays a crucial role in the training process of machine learning models, particularly neural networks. An epoch is one complete pass through the entire training dataset. The effect of the number of iterations on the estimate of a model's performance can vary based on several factors:

Underfitting and Overfitting:
Underfitting: If the number of iterations is too low, the model might not have enough chances to learn from the data. This can lead to underfitting, where the model is too simplistic and fails to capture the underlying patterns in the data.

Overfitting: On the other hand, if the number of iterations is too high, the model might start memorizing the training data instead of learning general patterns. This can lead to overfitting, where the model performs well on the training data but fails to generalize to new, unseen data.

The Goldilocks Principle:
The optimal number of iterations is often found through experimentation and is specific to the dataset and the complexity of the model. It's essential to strike a balance between underfitting and overfitting. This principle is sometimes called the Goldilocks Principle - not too few iterations (epochs), not too many, but just the right amount to capture the underlying patterns in the data without memorizing it.

Effect on Estimate:
Early Stopping: In practice, to avoid overfitting, techniques like early stopping are employed. Early stopping involves monitoring the model's performance on a validation dataset during training. Training is stopped when the performance on the validation dataset starts to degrade, indicating that the model has started to overfit.

Cross-Validation: Cross-validation can help estimate the optimal number of iterations. By evaluating the model's performance on different folds of the data, you can identify the number of iterations that results in the best average performance across different data subsets.

Learning Rate Schedules: The learning rate, which determines the size of the steps the model takes during training, can also impact the choice of the number of iterations. Learning rate schedules that adjust the learning rate during training can influence how quickly the model converges to a solution.

In summary, while increasing the number of iterations can sometimes improve the model's performance, it's not a guaranteed way to get a better estimate. The best approach is to experiment with different numbers of iterations, monitor the model's performance on validation data, and potentially use techniques like early stopping and cross-validation to find the optimal number of iterations for your specific model and dataset.

4.Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?

A.Increasing the number of iterations (epochs) might not be the most effective solution for dealing with a very small training dataset or validation dataset. When the dataset is small, it generally means there isn't enough diverse information for the model to learn robust patterns. Increasing the number of iterations without addressing the lack of data can lead to overfitting, where the model memorizes the training data instead of learning meaningful patterns. This can result in poor generalization to new, unseen data.

Challenges with a Small Dataset:
Limited Information: A small dataset often lacks the diversity and complexity required for the model to capture the underlying patterns in the data.

Overfitting Risk: With a small dataset, there is a higher risk of overfitting. The model might fit the noise in the data rather than the actual patterns, especially if the number of iterations is increased without proper constraints.

Better Approaches for Small Datasets:
Data Augmentation: If applicable, you can artificially increase the size of your dataset through data augmentation techniques. For example, in image recognition tasks, you can rotate, flip, or crop images to create variations of the original data.

Regularization Techniques: Use techniques like dropout and L1/L2 regularization to prevent overfitting. Dropout randomly drops out a fraction of the neurons during training, preventing reliance on specific features. L1/L2 regularization adds a penalty term to the loss function, discouraging overly complex models.

Transfer Learning: If your problem is similar to a problem for which a pre-trained model exists, you can use transfer learning. Transfer learning involves using a pre-trained model as a starting point and fine-tuning it on your small dataset.

Simpler Models: With limited data, it's often better to use simpler models that are less prone to overfitting. Avoid complex architectures that have many parameters relative to the size of the dataset.

Cross-Validation: Cross-validation can help assess the performance of your model more reliably, even with a small dataset. It provides a better estimate of how well the model might generalize to new data.

Ensemble Methods: Combining predictions from multiple models (ensemble methods) can sometimes improve generalization, especially if each model in the ensemble is trained differently (e.g., with different subsets of the data).

In summary, while increasing the number of iterations might help the model fit the training data better, it's not a fundamental solution for dealing with a very small dataset. Proper data preprocessing, regularization techniques, and, if applicable, data augmentation are more effective strategies for improving the performance and generalizability of models trained on small datasets.




